In [1]:
import torch
import numpy as np
np.random.seed(1328)
torch.random.manual_seed(1328)


In [2]:
from torch import nn
class LeNet(nn.Module):
    def __init__(self):
        super(LeNet, self).__init__()
        #Definiamo il primo livello. Dobbiamo effettuare una convoluzione 2D (ovvero su immagini)
        #Utilizziamo il modulo Conv2d che prende in input:
        # - il numero di canali in input: 1 (si tratta di immagini in scala di grigio)
        # - il numero di canali in output: 6 (le mappe di feature)
        # - la dimensione del kernel: 5 (sta per "5 X 5")
        self.C1 = nn.Conv2d(1, 6, 5)
        #Definiamo il livello di subsampling. Questo viene implementato usando il modulo "AvgPool2d"
        #Il modulo richiede in input la dimensione dei neighborhood rispetto ai quali calcolare
        # i valori medi: 2
        self.S2 = nn.AvgPool2d(2)
        #Definiamo il livello C3 in maniera analoga a quanto fatto per il livello C1:
        self.C3 = nn.Conv2d(6, 16, 5)
        #Definiamo il successivo max pooling 2d
        self.S4 = nn.AvgPool2d(2)
        #Definiamo il primo layer FC
        self.F5 = nn.Linear(256, 120)
        #Definiamo il secondo layer FC
        self.F6 = nn.Linear(120, 84)
        #Definiamo il terzo layer FC
        self.F7 = nn.Linear(84, 10)
        #Definiamo inoltre un modulo per calcolare l'attivazione ReLU
        self.activation = nn.Tanh()
    def forward(self,x):
        #Applichiamo le diverse trasformazioni in cascata
        x = self.C1(x)
        x = self.S2(x)
        x = self.activation(x) #inseriamo le attivazioni ove opportuno
        x = self.C3(x)
        x = self.S4(x)
        x = self.activation(x) #inseriamo le attivazioni ove opportuno
        x = self.F5(x.view(x.shape[0],-1)) #dobbiamo effettuare un "reshape" del tensore
        x = self.activation(x)
        x = self.F6(x)
        x = self.activation(x)
        x = self.F7(x)
        return x

In [3]:
net = LeNet()
sum([p.numel() for p in net.parameters()])

44426

In [4]:
from torchvision.datasets import MNIST
from torch.utils.data import DataLoader
from torchvision import transforms
transform = transforms.Compose([transforms.ToTensor(),
transforms.Normalize((0.1307,), (0.3081,))])
mnist_train = MNIST(root='mnist',train=True, download=True, transform=transform)
mnist_test = MNIST(root='mnist',train=False, download=True, transform=transform)
mnist_train_loader = DataLoader(mnist_train, batch_size=512, num_workers=2, shuffle=True)
mnist_test_loader = DataLoader(mnist_test, batch_size=512, num_workers=2)

In [5]:
from sklearn.metrics import accuracy_score
from torch.optim import SGD
from torch.autograd import Variable
def train_classification(model, lr=0.01, epochs=20, momentum=0.9, \
    train_loader=mnist_train_loader, test_loader=mnist_test_loader):
    criterion = nn.CrossEntropyLoss()
    optimizer = SGD(model.parameters(),lr, momentum=momentum)
    loaders = {'train':train_loader, 'test':test_loader}
    losses = {'train':[], 'test':[]}
    accuracies = {'train':[], 'test':[]}
    if torch.cuda.is_available():
        model=model.cuda()
    for e in range(epochs):
        print("Primo ciclo for.")
        for mode in ['train', 'test']:
            print("Secondo ciclo for.")
            if mode=='train':
                model.train()
            else:
                model.eval()
            epoch_loss = 0
            epoch_acc = 0
            samples = 0
            print("Mode-->",mode)
            print("Enumerate-->", loaders[mode])
            for i, batch in enumerate(loaders[mode]):
                #trasformiamo i tensori in variabili
                x=Variable(batch[0], requires_grad=(mode=='train'))
                #print("x shape-->",x.shape)
                y=Variable(batch[1])
                if torch.cuda.is_available():
                    x, y = x.cuda(), y.cuda()
                output = model(x)
                l = criterion(output,y)
                if mode=='train':
                    l.backward()
                    optimizer.step()
                    optimizer.zero_grad()
                print("L-->",l.item())
                acc = accuracy_score(y.cpu().data,output.cpu().max(1)[1].data)
                epoch_loss+=l.data.item()*x.shape[0]
                epoch_acc+=acc*x.shape[0]
                samples+=x.shape[0]
                print ("\r[%s] Epoch %d/%d. Iteration %d/%d. Loss: %0.2f. Accuracy: %0.2f\t\t\t\t\t" % \
                (mode, e+1, epochs, i, len(loaders[mode]), epoch_loss/samples, epoch_acc/samples),
                epoch_loss/samples,
                epoch_acc/samples,
                losses[mode].append(epoch_loss))
                accuracies[mode].append(epoch_acc)
            print("Fine secondo ciclo for")
        print("\r[%s] Epoch %d/%d. Iteration %d/%d. Loss: %0.2f. Accuracy: %0.2f\t\t\t\t\t" % \
        (mode, e+1, epochs, i, len(loaders[mode]), epoch_loss, epoch_acc))

    print("Ho finito.")
    #restituiamo il modello e i vari log
    return model, (losses, accuracies)

In [14]:
from torchvision.datasets import CIFAR100
from torch.utils.data import DataLoader
from torchvision import transforms
transform = transforms.Compose([transforms.ToTensor(),
transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
                               transforms.Resize(512,512)])
cifar_train = CIFAR100(root='cifar100',train=True, download=True, transform=transform)
cifar_test = CIFAR100(root='cifar100',train=False, download=True, transform=transform)
cifar_train_loader = DataLoader(cifar_train, batch_size=512, num_workers=2, shuffle=True)
cifar_test_loader = DataLoader(cifar_test, batch_size=512, num_workers=2)

Files already downloaded and verified
Files already downloaded and verified


In [7]:
from torch import nn
class LeNetColor(nn.Module):
    def __init__(self):
        super(LeNetColor, self).__init__()
        #ridefiniamo il modello utilizzando i moduli sequential.
        #ne definiamo due: un "feature extractor", che estrae le feature maps
        #e un "classificatore" che implementa i livelly FC
        self.feature_extractor = nn.Sequential(
        nn.Conv2d(3, 18, 5), #Input: 3 x 32 x 32. Ouput: 18 x 28 x 28
        nn.MaxPool2d(2), #Input: 18 x 28 x 28. Output: 18 x 14 x 14
        nn.ReLU(),
        nn.Conv2d(18, 28, 5), #Input 18 x 14 x 14. Output: 28 x 10 x 10
        nn.MaxPool2d(2), #Input 28 x 10 x 10. Output: 28 x 5 x 5
        nn.ReLU()
        )
        self.classifier = nn.Sequential(
        nn.Linear(700, 360), #Input: 28 * 5 * 5
        nn.ReLU(),
        nn.Linear(360, 252),
        nn.ReLU(),
        nn.Linear(252, 100)
        )
    def forward(self,x):
        #Applichiamo le diverse trasformazioni in cascata
        x = self.feature_extractor(x)
        x = self.classifier(x.view(x.shape[0],-1))
        return x

In [10]:
lenet_cifar = LeNetColor()
print(sum([p.numel() for p in lenet_cifar.parameters()]))
lenet_cifar, lenet_cifar_logs = train_classification(lenet_cifar, train_loader=cifar_train_loader, \
test_loader=cifar_test_loader, epochs=25)

382628
Primo ciclo for.
Secondo ciclo for.
Mode--> train
Enumerate--> <torch.utils.data.dataloader.DataLoader object at 0x000001B33D414828>
L--> 4.607678413391113
[train] Epoch 1/25. Iteration 0/98. Loss: 4.61. Accuracy: 0.01					 4.607678413391113 0.0078125 None
L--> 4.606502056121826
[train] Epoch 1/25. Iteration 1/98. Loss: 4.61. Accuracy: 0.01					 4.60709023475647 0.009765625 None
L--> 4.604459285736084
[train] Epoch 1/25. Iteration 2/98. Loss: 4.61. Accuracy: 0.01					 4.606213251749675 0.009114583333333334 None
L--> 4.606632232666016
[train] Epoch 1/25. Iteration 3/98. Loss: 4.61. Accuracy: 0.01					 4.60631799697876 0.01025390625 None
L--> 4.603316307067871
[train] Epoch 1/25. Iteration 4/98. Loss: 4.61. Accuracy: 0.01					 4.605717658996582 0.009765625 None
L--> 4.609220504760742
[train] Epoch 1/25. Iteration 5/98. Loss: 4.61. Accuracy: 0.01					 4.606301466623942 0.009114583333333334 None
L--> 4.604864120483398
[train] Epoch 1/25. Iteration 6/98. Loss: 4.61. Accuracy: 0.01			

[train] Epoch 1/25. Iteration 60/98. Loss: 4.60. Accuracy: 0.01					 4.602924417276851 0.01011782786885246 None
L--> 4.59523868560791
[train] Epoch 1/25. Iteration 61/98. Loss: 4.60. Accuracy: 0.01					 4.602800453862836 0.010049143145161291 None
L--> 4.594935417175293
[train] Epoch 1/25. Iteration 62/98. Loss: 4.60. Accuracy: 0.01					 4.602675612010653 0.010075644841269842 None
L--> 4.597671985626221
[train] Epoch 1/25. Iteration 63/98. Loss: 4.60. Accuracy: 0.01					 4.602597430348396 0.010101318359375 None
L--> 4.594735622406006
[train] Epoch 1/25. Iteration 64/98. Loss: 4.60. Accuracy: 0.01					 4.602476479456975 0.010036057692307693 None
L--> 4.595958232879639
[train] Epoch 1/25. Iteration 65/98. Loss: 4.60. Accuracy: 0.01					 4.602377718145197 0.010002367424242424 None
L--> 4.592299461364746
[train] Epoch 1/25. Iteration 66/98. Loss: 4.60. Accuracy: 0.01					 4.602227296402205 0.010086287313432836 None
L--> 4.594513893127441
[train] Epoch 1/25. Iteration 67/98. Loss: 4.60. Accu

[train] Epoch 2/25. Iteration 0/98. Loss: 4.57. Accuracy: 0.02					 4.570744037628174 0.017578125 None
L--> 4.570209503173828
[train] Epoch 2/25. Iteration 1/98. Loss: 4.57. Accuracy: 0.02					 4.570476770401001 0.021484375 None
L--> 4.553472995758057
[train] Epoch 2/25. Iteration 2/98. Loss: 4.56. Accuracy: 0.02					 4.5648088455200195 0.024088541666666668 None
L--> 4.562331199645996
[train] Epoch 2/25. Iteration 3/98. Loss: 4.56. Accuracy: 0.02					 4.564189434051514 0.0234375 None
L--> 4.5568671226501465
[train] Epoch 2/25. Iteration 4/98. Loss: 4.56. Accuracy: 0.03					 4.562724971771241 0.025 None
L--> 4.566650390625
[train] Epoch 2/25. Iteration 5/98. Loss: 4.56. Accuracy: 0.02					 4.563379208246867 0.0234375 None
L--> 4.552520275115967
[train] Epoch 2/25. Iteration 6/98. Loss: 4.56. Accuracy: 0.02					 4.5618279320853095 0.022879464285714284 None
L--> 4.548472881317139
[train] Epoch 2/25. Iteration 7/98. Loss: 4.56. Accuracy: 0.02					 4.560158550739288 0.022216796875 None
L-->

[train] Epoch 2/25. Iteration 61/98. Loss: 4.43. Accuracy: 0.04					 4.4348008555750695 0.040858114919354836 None
L--> 4.170652389526367
[train] Epoch 2/25. Iteration 62/98. Loss: 4.43. Accuracy: 0.04					 4.430608022780645 0.041294642857142856 None
L--> 4.202734470367432
[train] Epoch 2/25. Iteration 63/98. Loss: 4.43. Accuracy: 0.04					 4.427047498524189 0.041656494140625 None
L--> 4.187463760375977
[train] Epoch 2/25. Iteration 64/98. Loss: 4.42. Accuracy: 0.04					 4.423361594860371 0.04200721153846154 None
L--> 4.282200813293457
[train] Epoch 2/25. Iteration 65/98. Loss: 4.42. Accuracy: 0.04					 4.421222795139659 0.04219933712121212 None
L--> 4.12747859954834
[train] Epoch 2/25. Iteration 66/98. Loss: 4.42. Accuracy: 0.04					 4.416838553414416 0.04253148320895522 None
L--> 4.121346950531006
[train] Epoch 2/25. Iteration 67/98. Loss: 4.41. Accuracy: 0.04					 4.412493088666131 0.04296875 None
L--> 4.1869049072265625
[train] Epoch 2/25. Iteration 68/98. Loss: 4.41. Accuracy: 0.04

[train] Epoch 3/25. Iteration 2/98. Loss: 4.04. Accuracy: 0.08					 4.038004398345947 0.080078125 None
L--> 4.0005388259887695
[train] Epoch 3/25. Iteration 3/98. Loss: 4.03. Accuracy: 0.08					 4.028638005256653 0.08349609375 None
L--> 3.997279167175293
[train] Epoch 3/25. Iteration 4/98. Loss: 4.02. Accuracy: 0.08					 4.022366237640381 0.082421875 None
L--> 3.944572687149048
[train] Epoch 3/25. Iteration 5/98. Loss: 4.01. Accuracy: 0.09					 4.009400645891826 0.08626302083333333 None
L--> 4.024084091186523
[train] Epoch 3/25. Iteration 6/98. Loss: 4.01. Accuracy: 0.09					 4.0114982809339255 0.08510044642857142 None
L--> 3.9602816104888916
[train] Epoch 3/25. Iteration 7/98. Loss: 4.01. Accuracy: 0.09					 4.005096197128296 0.08740234375 None
L--> 3.875338554382324
[train] Epoch 3/25. Iteration 8/98. Loss: 3.99. Accuracy: 0.09					 3.9906786812676325 0.08897569444444445 None
L--> 4.0457444190979
[train] Epoch 3/25. Iteration 9/98. Loss: 4.00. Accuracy: 0.09					 3.9961852550506594 0

...

[train] Epoch 23/25. Iteration 80/98. Loss: 1.78. Accuracy: 0.52					 1.7804165946112738 0.5152874228395061 None
L--> 1.779090166091919
[train] Epoch 23/25. Iteration 81/98. Loss: 1.78. Accuracy: 0.52					 1.7804004186537208 0.5150533536585366 None
L--> 1.8782469034194946
[train] Epoch 23/25. Iteration 82/98. Loss: 1.78. Accuracy: 0.51					 1.7815792919641518 0.5147778614457831 None
L--> 1.7842164039611816
[train] Epoch 23/25. Iteration 83/98. Loss: 1.78. Accuracy: 0.51					 1.7816106861545926 0.5146484375 None
L--> 1.8037233352661133
[train] Epoch 23/25. Iteration 84/98. Loss: 1.78. Accuracy: 0.51					 1.7818708349676693 0.5145680147058823 None
L--> 1.8002276420593262
[train] Epoch 23/25. Iteration 85/98. Loss: 1.78. Accuracy: 0.51					 1.7820842862129211 0.5147619912790697 None
L--> 1.7668285369873047
[train] Epoch 23/25. Iteration 86/98. Loss: 1.78. Accuracy: 0.51					 1.7819089327735462 0.5149739583333334 None
L--> 1.849499225616455
[train] Epoch 23/25. Iteration 87/98. Loss: 1.78.

[train] Epoch 24/25. Iteration 20/98. Loss: 1.64. Accuracy: 0.55					 1.6399103119259788 0.5485491071428571 None
L--> 1.7521642446517944
[train] Epoch 24/25. Iteration 21/98. Loss: 1.65. Accuracy: 0.55					 1.6450127634135159 0.5481178977272727 None
L--> 1.5803930759429932
[train] Epoch 24/25. Iteration 22/98. Loss: 1.64. Accuracy: 0.55					 1.6422032117843628 0.549422554347826 None
L--> 1.729288101196289
[train] Epoch 24/25. Iteration 23/98. Loss: 1.65. Accuracy: 0.55					 1.645831748843193 0.54833984375 None
L--> 1.756298303604126
[train] Epoch 24/25. Iteration 24/98. Loss: 1.65. Accuracy: 0.55					 1.6502504110336305 0.545625 None
L--> 1.6814857721328735
[train] Epoch 24/25. Iteration 25/98. Loss: 1.65. Accuracy: 0.55					 1.651451771075909 0.5451472355769231 None
L--> 1.69362473487854
[train] Epoch 24/25. Iteration 26/98. Loss: 1.65. Accuracy: 0.55					 1.6530137326982286 0.5452835648148148 None
L--> 1.6255780458450317
[train] Epoch 24/25. Iteration 27/98. Loss: 1.65. Accuracy: 0.5

[train] Epoch 24/25. Iteration 80/98. Loss: 1.69. Accuracy: 0.54					 1.6934893661075168 0.5372781635802469 None
L--> 1.7291101217269897
[train] Epoch 24/25. Iteration 81/98. Loss: 1.69. Accuracy: 0.54					 1.6939237655662909 0.5372522865853658 None
L--> 1.7456425428390503
[train] Epoch 24/25. Iteration 82/98. Loss: 1.69. Accuracy: 0.54					 1.694546883364758 0.537132906626506 None
L--> 1.8525705337524414
[train] Epoch 24/25. Iteration 83/98. Loss: 1.70. Accuracy: 0.54					 1.6964281172979445 0.5368768601190477 None
L--> 1.7734332084655762
[train] Epoch 24/25. Iteration 84/98. Loss: 1.70. Accuracy: 0.54					 1.6973340595469755 0.5367876838235294 None
L--> 1.811928153038025
[train] Epoch 24/25. Iteration 85/98. Loss: 1.70. Accuracy: 0.54					 1.6986665490061739 0.5368595566860465 None
L--> 1.8969848155975342
[train] Epoch 24/25. Iteration 86/98. Loss: 1.70. Accuracy: 0.54					 1.7009460693118217 0.5365481321839081 None
L--> 1.7807650566101074
[train] Epoch 24/25. Iteration 87/98. Loss: 

[train] Epoch 25/25. Iteration 20/98. Loss: 1.57. Accuracy: 0.57					 1.5675568353562128 0.5710565476190477 None
L--> 1.6611499786376953
[train] Epoch 25/25. Iteration 21/98. Loss: 1.57. Accuracy: 0.57					 1.5718110691417346 0.5695134943181818 None
L--> 1.5777631998062134
[train] Epoch 25/25. Iteration 22/98. Loss: 1.57. Accuracy: 0.57					 1.5720698574314946 0.5696331521739131 None
L--> 1.5227631330490112
[train] Epoch 25/25. Iteration 23/98. Loss: 1.57. Accuracy: 0.57					 1.5700154105822246 0.5703938802083334 None
L--> 1.5460008382797241
[train] Epoch 25/25. Iteration 24/98. Loss: 1.57. Accuracy: 0.57					 1.5690548276901246 0.57 None
L--> 1.5873194932937622
[train] Epoch 25/25. Iteration 25/98. Loss: 1.57. Accuracy: 0.57					 1.569757314828726 0.5694861778846154 None
L--> 1.6106088161468506
[train] Epoch 25/25. Iteration 26/98. Loss: 1.57. Accuracy: 0.57					 1.5712703333960638 0.5687210648148148 None
L--> 1.5974754095077515
[train] Epoch 25/25. Iteration 27/98. Loss: 1.57. Accura

[train] Epoch 25/25. Iteration 80/98. Loss: 1.62. Accuracy: 0.56					 1.6164976638040425 0.5567853009259259 None
L--> 1.7357652187347412
[train] Epoch 25/25. Iteration 81/98. Loss: 1.62. Accuracy: 0.56					 1.617952146181246 0.5565929878048781 None
L--> 1.570698618888855
[train] Epoch 25/25. Iteration 82/98. Loss: 1.62. Accuracy: 0.56					 1.6173828265753136 0.5564759036144579 None
L--> 1.6266659498214722
[train] Epoch 25/25. Iteration 83/98. Loss: 1.62. Accuracy: 0.56					 1.6174933399472917 0.5563848586309523 None
L--> 1.6938060522079468
[train] Epoch 25/25. Iteration 84/98. Loss: 1.62. Accuracy: 0.56					 1.618391136562123 0.5560661764705882 None
L--> 1.6677756309509277
[train] Epoch 25/25. Iteration 85/98. Loss: 1.62. Accuracy: 0.56					 1.6189653748689696 0.555686773255814 None
L--> 1.5874704122543335
[train] Epoch 25/25. Iteration 86/98. Loss: 1.62. Accuracy: 0.56					 1.6186033638044335 0.5558997844827587 None
L--> 1.67745840549469
[train] Epoch 25/25. Iteration 87/98. Loss: 1.6

In [12]:
plot_logs_classification(lenet_cifar_logs)

NameError: name 'plot_logs_classification' is not defined

In [15]:
from AlexNet import *
net = AlexNet()
sum([p.numel() for p in net.parameters()])
#lenet_cifar = LeNetColor()
#print(sum([p.numel() for p in lenet_cifar.parameters()]))
lenet_cifar, lenet_cifar_logs = train_classification(net, train_loader=cifar_train_loader, \
test_loader=cifar_test_loader, epochs=1)

Primo ciclo for.
Secondo ciclo for.
Mode--> train
Enumerate--> <torch.utils.data.dataloader.DataLoader object at 0x000001B30205EE10>


TypeError: Traceback (most recent call last):
  File "C:\Users\franc\Anaconda3\lib\site-packages\torch\utils\data\_utils\worker.py", line 99, in _worker_loop
    samples = collate_fn([dataset[i] for i in batch_indices])
  File "C:\Users\franc\Anaconda3\lib\site-packages\torch\utils\data\_utils\worker.py", line 99, in <listcomp>
    samples = collate_fn([dataset[i] for i in batch_indices])
  File "C:\Users\franc\Anaconda3\lib\site-packages\torchvision\datasets\cifar.py", line 127, in __getitem__
    img = self.transform(img)
  File "C:\Users\franc\Anaconda3\lib\site-packages\torchvision\transforms\transforms.py", line 61, in __call__
    img = t(img)
  File "C:\Users\franc\Anaconda3\lib\site-packages\torchvision\transforms\transforms.py", line 196, in __call__
    return F.resize(img, self.size, self.interpolation)
  File "C:\Users\franc\Anaconda3\lib\site-packages\torchvision\transforms\functional.py", line 229, in resize
    raise TypeError('img should be PIL Image. Got {}'.format(typ